In [1]:
import re
import json
import pickle
import numpy as np
import pandas as pd

from glob import glob

In [2]:
total = 0
without_hess = 0
for file in glob('results/optimizer/*'):
    total += 1
    with open(file, 'rb') as f:
        res = pickle.load(f)

    without_hess += res.hess_inv is None

without_hess / total

0.5716307039264238

In [3]:
df_results = pd.DataFrame(columns = ['competition', 'year', 'model', 'fitted_games', 'pars',
                                     'home', 'away', 'result', 'deFinetti', 'prob_home', 'prob_draw', 'prob_away'])

for file in glob('results/probs/game_probs*'):
    competition, year, model, games, pars = re.findall('game_probs_(Serie_[AB])_(\d+)_(.+)_(\d+)_games_(\d+)_pars', file)[0]
    games = int(games)
    if games == 380: continue
    with open(file, 'r') as f: probs = json.load(f)
    with open(f'../data/BrazilianSoccerData/results/processed/{competition}_{year}_games.json', 'r') as f: data = json.load(f)
    year = int(year)
    pars = int(pars)
    for game in range(games + 1, games + 11):
        game = f'{game}'
        if game not in data or 'Home' not in data[game]: continue
        home = data[game]['Home']
        away = data[game]['Away']
        result = data[game]['Result'].upper().split(' X ')
        result[0], result[1] = int(result[0]), int(result[1])
        if result[0] > result[1]: result, result_vector = 'H', np.array([1, 0, 0])
        elif result[0] == result[1]: result, result_vector = 'D', np.array([0, 1, 0])
        else: result, result_vector = 'A', np.array([0, 0, 1])
        prob_home = np.sum(np.tril(probs[home][away], -1))
        prob_draw = np.sum(np.diag(probs[home][away]))
        prob_away = np.sum(np.triu(probs[home][away], 1))
        probs_vector = np.array([prob_home, prob_draw, prob_away])
        definetti = np.sum((probs_vector - result_vector) ** 2)
        prob_result = probs_vector[np.argmax(result_vector)]
        df_results.loc[len(df_results)] = competition, year, model, games, pars, home, away, result, definetti, prob_home, prob_draw, prob_away

definetti_metric = df_results[['model', 'deFinetti', 'pars']].groupby(['model', 'pars']).mean().reset_index()
definetti_metric.loc[len(definetti_metric)] = ['baseline', '0', np.sum((1/3 * np.ones(3) - result_vector) ** 2)]
definetti_metric.sort_values('deFinetti', ignore_index=True)

,model,pars,deFinetti
0,independents_poisson,41,0.626369
1,shock_model,61,0.629816
2,independents_poisson,60,0.633856
3,shock_model,80,0.638429
4,holgates_poisson,61,0.641638
5,independents_poisson,80,0.649692
6,shock_model,100,0.651488
7,holgates_poisson,80,0.654884
8,independents_poisson,40,0.662361
9,shock_model,60,0.665527
